# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import numpy as np
import gmaps
import requests


# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachny,66.4167,112.4000,-39.20,72,98,1.19,RU,1701731053
1,1,agat village,13.3886,144.6585,29.42,82,100,8.23,GU,1701731054
2,2,bhimunipatnam,17.8833,83.4333,23.24,92,100,5.75,IN,1701731055
3,3,port mathurin,-19.6833,63.4167,25.76,82,4,3.61,MU,1701731056
4,4,stanley,54.8680,-1.6985,5.43,93,75,5.14,GB,1701730778


# Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
gmaps.configure(api_key=geoapify_key)

#get locations and humidity
city_locations = city_data_df[['Lat', 'Lng']].astype(float)
humidity = city_data_df['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px s…

In [10]:
data_plot = city_data_df.hvplot.points('Lat', 'Lng', size = "Humidity", color="City")
data_plot


:Points   [Lat,Lng]   (City,Humidity)

# DataFrame with ideal weather condition

In [4]:
# Naaarrow down cities that fit criteria and drop any results with null values
myFavoriteWeather = city_data_df.loc[
    (city_data_df['Max Temp'] >= 21) & 
    (city_data_df['Max Temp'] <= 32) & 
    (city_data_df['Wind Speed'] <= 20) & 
    (city_data_df['Humidity'] <= 40) & 
    (city_data_df['Cloudiness'] <= 50)
]

myFavoriteWeather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,ulladulla,-35.3500,150.4667,28.76,39,39,2.53,AU,1701731070
278,278,san rafael,-34.6177,-68.3301,24.92,20,5,5.35,AR,1701731168
346,346,santiago,-33.4569,-70.6483,24.69,30,0,5.66,CL,1701731197
390,390,manga,11.6636,-1.0731,25.95,17,39,3.50,BF,1701731215
435,435,alice springs,-23.7000,133.8833,29.75,26,0,2.57,AU,1701730972
468,468,alaghsas,17.0187,8.0168,22.96,17,23,5.16,NE,1701731246
476,476,nuriootpa,-34.4833,139.0000,25.77,25,47,1.86,AU,1701731250
559,559,ouahigouya,13.5828,-2.4216,25.24,18,14,4.03,BF,1701731292


# Create a new DataFrame called hotel_df

In [5]:
# Create hotel_df using the copy function
hotel_df = myFavoriteWeather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column "Hotel Name"
hotel_df['Hotel Name'] = ""

# Reset the index
hotel_df.reset_index(drop=True, inplace=True)

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,ulladulla,AU,-35.3500,150.4667,39,
1,san rafael,AR,-34.6177,-68.3301,20,
2,santiago,CL,-33.4569,-70.6483,30,
3,manga,BF,11.6636,-1.0731,17,
4,alice springs,AU,-23.7000,133.8833,26,
5,alaghsas,NE,17.0187,8.0168,17,
6,nuriootpa,AU,-34.4833,139.0000,25,
7,ouahigouya,BF,13.5828,-2.4216,18,


In [6]:
import requests

# Set parameters to search for a hotel
radius = 1000000
params = {
    'type': 'hotel',
    'radius': radius,
    'limit': 1,
    'apiKey': 'YOUR_API_KEY'}  

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{radius}@{lng},{lat}'
    params['bias'] = f'circle:{radius}@{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    result = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = result['features'][0]['properties']['name']
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)


Starting hotel search
ulladulla - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found
santiago - nearest hotel: No hotel found
manga - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
alaghsas - nearest hotel: No hotel found
nuriootpa - nearest hotel: No hotel found
ouahigouya - nearest hotel: No hotel found
            City Country      Lat       Lng  Humidity      Hotel Name
0      ulladulla      AU -35.3500  150.4667        39  No hotel found
1     san rafael      AR -34.6177  -68.3301        20  No hotel found
2       santiago      CL -33.4569  -70.6483        30  No hotel found
3          manga      BF  11.6636   -1.0731        17  No hotel found
4  alice springs      AU -23.7000  133.8833        26  No hotel found
5       alaghsas      NE  17.0187    8.0168        17  No hotel found
6      nuriootpa      AU -34.4833  139.0000        25  No hotel found
7     ouahigouya      BF  13.5828   -2.4216        18  No hotel found
